In [1]:
from test_data_generator import generate_fake_metro, plot_metro_layout
import random
import pandas as pd

flow_paths, stations, df = generate_fake_metro(
    station_count=10,
    flow_path_count=10,
    max_flow_path_length=5,
    min_flow_path_frequency=1,
    max_flow_path_frequency=10
)

test_stations = stations
test_paths = flow_paths

In [2]:
def generate_slots(test_paths):
    """
    Generates slots given the paths
    """
    max_length = int(len(test_paths) / 2) + 1
    list_of_list = [[(i,i), (i, -i), (-i, i), (-i, -i)] for i in range (1, max_length)]

    flattened_list = [(0, 0)]

    for sublist in list_of_list:
        for tuple_item in sublist:
            flattened_list.append(tuple_item)
    slot_names = ['s{}'.format(i) for i in range(0, len(flattened_list))]
    return list(zip(slot_names, flattened_list))


In [3]:
from algo import combination_to_coordinates, count_intersections
from utils import Point, LayoutAlgorithm, FlowPathsT, LayoutOutput, SLOTS
import numpy as np

def calculate_intersections(flow_paths, stations):
    """
    Takes a station, the flowpaths with frequencies and computes the optimal arrangement
    """

    slots = generate_slots(test_paths)

    data_paths = {
    'Paths': [n[1] for n in flow_paths],
    'Frequency': [n[0] for n in flow_paths],
    'Slot offsets': random.sample(slots, 10)
    }

    df_paths = pd.DataFrame(data_paths)

    # Transform offsets to a dictionary
    OFFSETS = df_paths['Slot offsets']
    offset_dict = dict()
    for offset in OFFSETS:
        offset_dict.update({offset[0]: offset[1]})

    # Get slot names
    slot_names = list(offset_dict.keys())

    SLOT_OFFSETS = offset_dict
    SLOTS = slot_names

    # Get slot names
    slot_coordinates = {}

    #Transforms the slot to a location
    for station_name, point in stations.items():
        for slot in SLOTS:
            offset_x, offset_y = SLOT_OFFSETS[slot]
            slot_coordinates[(station_name, slot)] = Point(point.x + offset_x, point.y + offset_y)

    combination_merged = []

    # For each path in the dataframe, it takes the slot offset and takes the name.
    # Then for each element of the path it creates a tuple with both.
    for path in df_paths.iterrows():
        path_list = []
        slot = path[1]['Slot offsets'][0]
        for station in path[1]['Paths']:
            path_list.append((station, slot))
        combination_merged.append(path_list)
    intersections = count_intersections(combination_to_coordinates(combination_merged, slot_coordinates))
    layout = list(map(lambda x: (1, x), combination_to_coordinates(combination_merged, slot_coordinates)))
    return intersections, combination_merged, layout

intersections_test , combination_merged , layout= calculate_intersections(test_paths, test_stations)
intersections_test

37

In [5]:
def dynamic_ranges(self, flow_paths, stations):
    """
    Takes the coordinates of a station, the paths between computes the optimal arrangement
    """
    #TODO: keep track of used combinations

    paths = flow_paths
    # Initialize iteration counter
    iteration = 0
    
    # Initialize dictionary with lowest scores
    amount_of_lower_scores = 100
    lowest_scores = {key: 10 for key in range(1000, (1000 + amount_of_lower_scores))}

    # Initialize score ranges
    score_range = 0

    # Compute score range
    search_space = len(paths) * len(paths) * 2 * len(paths)

    # Find the lowest score while ranges don't match
    while max(lowest_scores) > score_range and len(lowest_scores) > 1:

        # Calculate intersections
        intersection_value, combinations, layout  = calculate_intersections(flow_paths, stations)

        # Calculate overlap
        if intersection_value < max(lowest_scores.keys()):
            lowest_scores.pop(max(lowest_scores))
            lowest_scores.update({intersection_value: (combinations, layout)})

        # Decrease size of lowest scores list -> max(lowest scores) becomes smaller
        if iteration in [round(x) for x in np.linspace(1, search_space)]:
            lowest_scores.pop(max(lowest_scores))

        # Increase the score_range
        if iteration in [round(x) for x in np.linspace(1, search_space)]:
            score_range += 1

        iteration += 1
 

    best_score =  min(lowest_scores, key = lowest_scores.get), lowest_scores[min(lowest_scores, key = lowest_scores.get)]
    return best_score[0], best_score[1][0], best_score[1][1]


intersections, paths, layout= dynamic_ranges([], test_paths, test_stations)
intersections

20